# This is how to manage PROCESSES and JOBS with the Playground API
This Notebook shows how to connect to the Intelligence Playground Python Client API and do amazing stuff!

In [1]:
# Install some extra python module if needed...
#!pip install mercantile geohash2 geojson shapely ipywidgets ipyleaflet

In [2]:
# Standard imports
import os
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import numpy as np
from skimage import exposure, color, img_as_ubyte
import mercantile
import requests
import csv
import uuid
import json
import geohash2
import uuid
import warnings
import geojson
import time
warnings.filterwarnings("ignore")

from shapely.geometry import mapping, shape

# Widgets imports
from ipyleaflet import Map, TileLayer, Polygon
import ipywidgets as widgets
from IPython.core.display import HTML

## Connecting to the API

To connect to the API, you need to retrieve your API_KEYS from the OneAtlas website. Follow the simple steps below:

1. Visit this URL: https://data.api.oneatlas.airbus.com/api-keys
2. Click the **Create and API key** button
3. Enter a description for your API_KEY (e.g. Playground Keys)
4. Store the file in the same folder than this notebook and name it **api_key.txt**

Make sure to keep your **api_key.txt** safe! Do not include it in a public github repository for example :-)

The following script will then use this **api_key.txt** file to generate an ACCESS_TOKEN. We will store this ACCESS_TOKEN in HEADERS that we will send with each requests. The ACCESS_TOKEN has a timeout so we will create a function that renew the ACCESS_TOKEN when half of the timeout has expired. 

In [3]:
# Create a PlaygroundClient object
from playgroundclient import PlaygroundClient
play = PlaygroundClient()

## Some useful functions

In [4]:
def print_json(parsed):
    print(json.dumps(parsed, indent=4, sort_keys=True))
    
def getBounds(coords):
    xmax = ymax = -np.Infinity
    xmin = ymin = np.Infinity
    for v in coords:
        xmax = xmax if xmax > v[0] else v[0]
        ymax = ymax if ymax > v[1] else v[1]
        xmin = xmin if xmin < v[0] else v[0]
        ymin = ymin if ymin < v[1] else v[1]
    return (xmin, ymin, xmax, ymax)


## Now start using them!

In [5]:
# Logged in user
user = play.get_logged_user()
print("Logged as user: {} {}".format(user['firstname'], user['lastname']))

Logged as user: Jean-Francois Faudi


## Get available projects for this user

In [6]:
p_database = {}
p_label = []
r = play.get_projects()
#print(r)
for project in r:
    #print(process)
    project_id = project['project_id']
    p_label.append((project['name'], project_id))
    p_database[project_id] = project
    
p_label.sort() 
p_w = widgets.Dropdown(
    options=p_label,
    description='Project:',
    disabled=False
)
display(p_w)

Dropdown(description='Project:', options=(('Airbus', '1a50823a-d9c1-4397-8b64-1c8079d6c7eb'), ('Airbus NA', 'e…

## Get available processes in this project

In [7]:
projectId = p_w.value

a_database = {}
a_label = []
r = play.get_processes(projectId=projectId)
#print(processes)
for process in r['processes']:
    #print(process)
    process_id = process['id']
    a_label.append((process['title'], process_id))
    a_database[process_id] = process
    
a_label.sort() 
a_w = widgets.Dropdown(
    options=a_label,
    description='Process:',
    disabled=False
)
display(a_w)

Dropdown(description='Process:', options=(('DL Change Detection 5', '2f8d2fc7-e38e-4e2f-88f2-266135677f4c'), (…

## Get more precise information about this process

In [8]:
processId = a_w.value
print("Process_ID={}".format(processId))
r = play.get_process(projectId, processId)
#print_json(r)

Process_ID=2f8d2fc7-e38e-4e2f-88f2-266135677f4c


### Description of input to process

In [9]:
print_json(r['input'])

{
    "$schema": "http://json-schema.org/draft-06/schema#",
    "description": "Geo Processes Manager API input schema for tile change detection",
    "oneOf": [
        {
            "required": [
                "datasetId"
            ]
        },
        {
            "required": [
                "datasetName"
            ]
        }
    ],
    "properties": {
        "datasetId": {
            "description": "Result dataset ID, if the dataset already exists (exclusive with datasetName)",
            "type": "string"
        },
        "datasetName": {
            "description": "Result dataset name, to create a new dataset (exclusive with datasetId)",
            "type": "string"
        },
        "geom": {
            "description": "Area of interest",
            "properties": {
                "coordinates": {
                    "description": "GeoJSON Polygon coordinates",
                    "type": "array"
                },
                "type": {
                    "

### Description of output of this process (not implemented)

In [10]:
# TODO: this field should contain information about the output format of the process
print_json(r['output'])

null


### Description of available actions on this process

In [11]:
print_json(r['_links'])

{
    "delete": {
        "href": "https://playground-api-gateway.playground.airbusds-geo.com/api/v1/processes/2f8d2fc7-e38e-4e2f-88f2-266135677f4c?projectId=3361252e-bfe8-4b8e-8c38-e915e32d741f",
        "method": "DELETE",
        "relation": "Delete the process",
        "type": "application/json"
    },
    "jobs": {
        "href": "https://playground-api-gateway.playground.airbusds-geo.com/api/v1/processes/2f8d2fc7-e38e-4e2f-88f2-266135677f4c/jobs?projectId=3361252e-bfe8-4b8e-8c38-e915e32d741f",
        "method": "GET",
        "relation": "Return the list of all process' jobs",
        "type": "application/json"
    },
    "list": {
        "href": "https://playground-api-gateway.playground.airbusds-geo.com/api/v1/processes?projectId=3361252e-bfe8-4b8e-8c38-e915e32d741f",
        "method": "GET",
        "relation": "Return the list of all processes",
        "type": "application/json"
    },
    "runNewJob": {
        "href": "https://playground-api-gateway.playground.airbusds-

## Searching for jobs

In [15]:
# Overloading the class
class MyPlaygroundClient(PlaygroundClient):
    def __init__(self):
        super().__init__()
    
    PLAYGROUND_JOBS_COUNT_URL = PlaygroundClient.PLAYGROUND_URL + "/api/jobs?project_id={projectId}&page_size={pageSize}&page={page}&filter_user=true&count=true"
    def get_jobs_count(self, projectId, pageSize=10, page=0):
        return self._get_request(self.PLAYGROUND_JOBS_COUNT_URL, projectId=projectId, pageSize=pageSize, page=page)

    PLAYGROUND_JOBS_URL = PlaygroundClient.PLAYGROUND_URL + "/api/jobs?project_id={projectId}&page_size={pageSize}&page={page}&filter_user=true"
    def get_jobs(self, projectId, pageSize=10, page=0):
        return self._get_request(self.PLAYGROUND_JOBS_URL, projectId=projectId, pageSize=pageSize, page=page)

In [16]:
myplay = MyPlaygroundClient()
r = myplay.get_jobs_count(projectId)
print("There are {} jobs for the selected project.".format(r['count']))

r = myplay.get_jobs(projectId)
#print_json(r)

job_database = {}
job_label = []
for job in r:
    job_id = job['job_id']
    job_label.append(("{} ({}) -> {} tiles".format(job["name"], job["process"]["name"], job["tiles_estimated"]), job_id))
    a_database[job_id] = job
    
job_label.sort() 
job_w = widgets.Dropdown(
    options=job_label,
    description='Jobs:',
    disabled=False
)
display(job_w)

There are 71 jobs for the selected project.


Dropdown(description='Jobs:', options=(('Doha (Ext Cars) -> 70 tiles', '1a2deece-27a8-46fe-8bc4-84f31179b305')…